In [1]:
import pickle
import json
import joblib
import pandas as pd
import category_encoders
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

from custom_transformers.transformer import StandardScaler

In [2]:
df = pd.read_csv('titanic.csv')
df = df.drop(['Ticket', 'Name', 'PassengerId'], axis=1)
X_train, y_train = df.drop('Survived', axis=1), df.Survived

In [3]:
scaler = StandardScaler()
scaler.fit(X_train)
scaler.transform(X_train).head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,-0.530005,1,0,-0.502163,NaN,S
1,1,female,0.571430,1,0,0.786404,C85,C
2,3,female,-0.254646,0,0,-0.488580,NaN,S
3,1,female,0.364911,1,0,0.420494,C123,S
4,3,male,0.364911,0,0,-0.486064,NaN,S


In [4]:
pipeline = make_pipeline(
    category_encoders.OneHotEncoder(),
    StandardScaler(),
    SimpleImputer(strategy='mean'),
    LogisticRegression(),
)
pipeline.fit(X_train, y_train)

/Users/chi/.local/share/virtualenvs/BLU13/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Pipeline(steps=[('onehotencoder',
                 OneHotEncoder(cols=['Sex', 'Cabin', 'Embarked'])),
                ('standardscaler',
                 <custom_transformers.transformer.StandardScaler object at 0x7fd9366b6e50>),
                ('simpleimputer', SimpleImputer()),
                ('logisticregression', LogisticRegression())])

In [5]:
new_obs_str = '{"Age": 22.0, "Cabin": null, "Embarked": "S", "Fare": 7.25, "Parch": 0, "Pclass": 3, "Sex": "male", "SibSp": 1}'
new_obs_dict = json.loads(new_obs_str)
obs = pd.DataFrame([new_obs_dict], columns=X_train.columns.tolist())
obs = obs.astype(X_train.dtypes)
obs

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,3,male,22.0,1,0,7.25,None,S


In [6]:
pipeline.predict_proba(obs)

array([[0.83900696, 0.16099304]])

**Serializing stuff**

In [7]:
with open('columns.json', 'w') as fh:
    json.dump(X_train.columns.tolist(), fh)
    
with open('dtypes.pickle', 'wb') as fh:
    pickle.dump(X_train.dtypes, fh)
    
joblib.dump(pipeline, 'pipeline.pickle') 

['pipeline.pickle']

**Deserializing stuff**

In [8]:
with open('columns.json', 'r') as fh:
    columns = json.load(fh)
    
with open('dtypes.pickle', 'rb') as fh:
    dtypes = pickle.load(fh)
    
recovered_pipeline = joblib.load('pipeline.pickle')

In [9]:
recovered_pipeline.predict_proba(obs)

array([[0.83900696, 0.16099304]])